In [97]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [98]:
def remove_dollar_sign_and_comma(cell):
    """
    Remove dollar signs ('$'), commas (',') and dashes/minuses ('-') or pluses ('+') from a given string representation of a number.

    Args:
        cell (str or any): The string or value from which to remove dollar signs and commas.

    Returns:
        str or any: The modified string with dollar signs and commas removed, or the original value if not a string.
    """
    if isinstance(cell, str):
        cell = cell.replace("$", "").replace(",", "").replace("-", "").replace("+", "")
    return cell

In [ ]:
# Function to validate and transform zip codes
def validate_and_transform(zip_code):
    # Check for 5 digits optionally followed by a dash and 4 more digits
    if pd.notna(zip_code) and (zip_code.isdigit() or (len(zip_code) == 10 and zip_code[5] == '-')):
        # Add leading zero if only 4 digits
        if len(zip_code) == 4:
            return '0' + zip_code
        if len(zip_code) == 3:
            return '00' + zip_code
        # Valid formats directly returned
        elif len(zip_code) == 5 or (len(zip_code) == 10 and zip_code.replace('-', '').isdigit()):
            return zip_code
    return None  # Return None for invalid zip codes

### **Read the MSA Data**

In [115]:
filename =  "ZipTables.xlsx"
df_msa = pd.read_excel(filename, sheet_name="Zip_MSA")
df_demo = pd.read_excel(filename, sheet_name="Zip_Demographics")

### **Rename the columns in the MSA Sheet**

In [116]:
# Define the new column names in snake case
new_column_names = {
    'ZIP CODE': 'zip',
    'STATE': 'msa_state',
    'MSA No.': 'msa_no',
    'MSA Name': 'msa_name'
}

# Rename the columns
df_msa.rename(columns=new_column_names, inplace=True)

### **Rename the columns in the Demographics Sheet**

In [117]:
# Define the new column names in snake case
new_column_names = {
    'Zip': 'zip',
    'Percent Population 25 years and over Less than 9th grade_raw': 'percent_less_than_9th_grade',
    'Estimate Percent Population 25 years and over 9th to 12th grade no diploma_raw': 'percent_9_to_12_no_diploma',
    'Percent Population 25 years and over Percent high school graduate or higher_raw': 'percent_hs_grad_or_higher',
    'Percent Population 25 years and over Percent bachelors degree or higher_raw': 'percent_bachelors_or_higher',
    'Owner-occupied housing units Percent_raw': 'owner_occupied_percent',
    'Median age_raw': 'median_age',
    'Median Income_raw': 'median_income',
    'Median Household Value_raw': 'median_household_value',
    'Labor Participation Rate_raw': 'labor_participation_rate'
}

# Rename the columns
df_demo.rename(columns=new_column_names, inplace=True)

### **Merge the two sheets on "zip"**

In [102]:
df_irs = df_demo.merge(df_msa, on="zip", how="left")

In [103]:
#df = df_irs[column_dtypes_float]
#df[df.apply(lambda row: row.astype(str).str.contains('-').any(), axis=1)]

### **Formatting and Editing**

In [104]:
df_irs["owner_occupied_percent"] = 100 * df_irs["owner_occupied_percent"]

In [105]:
### For median_age

# Replace '-' and '' with NaN
df_irs.replace({"median_age": {"-": np.nan, "": np.nan}}, inplace=True)

# Replace NaN with the median
df_irs["median_age"].fillna(df_irs["median_age"].median(), inplace=True)

### For everything else

# Replace '-' with 0
df_irs.replace({"-": np.nan, "": np.nan}, inplace=True)

# Replace NaN with 0
df_irs.fillna(0, inplace=True)

In [106]:
df_irs = df_irs.map(remove_dollar_sign_and_comma)

In [107]:
column_dtypes_float = ['percent_less_than_9th_grade', 'percent_9_to_12_no_diploma',
       'percent_hs_grad_or_higher', 'percent_bachelors_or_higher',
       'owner_occupied_percent', 'median_age', 'median_income',
       'median_household_value', 'labor_participation_rate']

# Convert specified columns to float
df_irs[column_dtypes_float] = df_irs[column_dtypes_float].astype(float)
df_irs["zip"] = df_irs["zip"].astype(str)

In [108]:
df_irs = df_irs[['zip', 'msa_state', 'msa_no', 'msa_name', 
       'percent_less_than_9th_grade', 'percent_9_to_12_no_diploma',
       'percent_hs_grad_or_higher', 'percent_bachelors_or_higher',
       'owner_occupied_percent', 'median_age', 'median_income',
       'median_household_value', 'labor_participation_rate']]

In [109]:
df_irs.head()

,zip,msa_state,msa_no,msa_name,percent_less_than_9th_grade,percent_9_to_12_no_diploma,percent_hs_grad_or_higher,percent_bachelors_or_higher,owner_occupied_percent,median_age,median_income,median_household_value,labor_participation_rate
0,601,PR,99072,PR NONMETROPOLITAN AREA,23.9,10.5,65.6,16.4,59.593393,41.9,14361.0,83900.0,40.2
1,602,PR,10380,AguadillaIsabelaSan Sebastian PR MSA,23.8,10.5,65.7,20.2,74.788069,42.9,16807.0,85300.0,39.7
2,603,PR,10380,AguadillaIsabelaSan Sebastian PR MSA,16.8,10.2,73.0,23.0,57.821926,42.1,16049.0,118400.0,39.1
3,606,PR,99072,PR NONMETROPOLITAN AREA,32.6,14.9,52.5,7.6,75.539568,44.3,12119.0,80800.0,26.0
4,610,PR,10380,AguadillaIsabelaSan Sebastian PR MSA,21.9,8.8,69.4,18.6,76.839113,42.7,19898.0,87600.0,41.3


In [110]:
df_irs.to_csv("irs_msa.csv")

In [111]:
df_irs.to_parquet("irs_msa.parquet.gzip", compression="gzip")  

In [112]:
df_irs.to_parquet("irs_msa.parquet")  

In [113]:
df = pd.read_parquet("irs_msa.parquet")